In [ ]:
from datascience import *
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

# Lecture 31

## Review: Predicting a child's height

Let's revisit Galton's predictions of children's heights based on their parent's heights...

In [ ]:
galton = Table.read_table('galton.csv')

heights = Table().with_columns(
    'MidParent', galton.column('midparentHeight'),
    'Child', galton.column('childHeight')
    )
heights

In [ ]:
def predict_child(h):
    """Return a prediction of the height of a child 
    whose parents have a midparent height of h.
    
    The prediction is the average height of the children 
    whose midparent height is in the range h plus or minus 0.25 inches.
    """
    
    close_points = heights.where('MidParent', are.between(h-0.5, h + 0.5))
    return close_points.column('Child').mean()   

In [ ]:
# predict the height for each child in the data set
heights_with_predictions = heights.with_column(
    'Prediction', heights.apply(predict_child, 'MidParent')
)

In [ ]:
# visualize the predicted heights
heights_with_predictions.scatter('MidParent')

### Review: The correlation coefficient

In [ ]:
# Convert data to standard units
def standard_units(x):
    "Convert any array of numbers to standard units."
    return (x - np.average(x))/np.std(x)  

In [ ]:
# we can create a function to calculate the correlation coefficient
def correlation(t, label_x, label_y):
    x_in_standard_units = standard_units(t.column(label_x))
    y_in_standard_units = standard_units(t.column(label_y))
    return np.sum(x_in_standard_units * y_in_standard_units)/(t.num_rows - 1)

In [ ]:
# correlation of Galton heights
correlation(heights, 'MidParent', 'Child')

## Linear regression

In [ ]:
# original scatter plot of the data and the correlation for the Galton data
heights.scatter('MidParent')

In [ ]:
# predictions made by taking average of children's heights in a neighborhood
heights_with_predictions.scatter('MidParent')

In [ ]:
# predictions made by the regression line
heights.scatter('MidParent', fit_line = True)

In [ ]:
# comparing prediction of the regression line and average in a neighborhood
heights_with_predictions.scatter('MidParent', fit_line = True)

## Regression in standardized units

In [ ]:
# Let's look at the relationship in standaridized units (z-score transformed units)

heights_standardized = Table().with_columns(
    "MidParent", standard_units(heights.column("MidParent")),
    "Child", standard_units(heights.column("Child"))
)

heights_standardized

In [ ]:
heights_standardized.scatter("MidParent", fit_line = True)

In [ ]:
# Correlation between children's and parent's heights
correlation(heights, "Child", "MidParent")

In [ ]:
# predictions are less than the identity line -> regression to the mean
heights_standardized.scatter("MidParent", fit_line = True)
plots.plot(make_array(-3, 3), make_array(-3, 3), color="red");

In [ ]:
# function to calculate the slope 
def slope(t, x, y):
    r = correlation(t, x, y)
    y_sd = np.std(t.column(y))
    x_sd = np.std(t.column(x))
    return r * y_sd / x_sd


In [ ]:
# slope for predicting child's height
slope(heights, "MidParent", "Child")

# Q: for every additional inch a parent is taller, how much taller is the predicted child's height?

In [ ]:
# function to calculate the intercept
def intercept(t, x, y):
    x_mean = np.mean(t.column(x))
    y_mean = np.mean(t.column(y))
    return y_mean - slope(t, x, y)*x_mean


In [ ]:
# intercept for predicting child's height
intercept(heights, "MidParent", "Child")

#Q: How a parents that are 0" tall, how tall is the predicted height of their child? 

### Regression equation for Galton data

$\hat{y} = 0.638 \cdot x + 22.589$



In [ ]:
# How tall would be predict a child to be if their parents were 70 inches?
x = 72
0.638 * x + 22.589

## Regression Line 

In [ ]:
# Helper function: Draw a line for a given slope, intercept, and possibly input location x
def draw_line(slope=0, intercept=0, x=None, color='r'):
    if x is None:
        x1, x2, y1, y2 = plots.gca().axis()
    x = make_array(x1, x2)
    y = x*slope + intercept
    plots.plot(x, y, color=color)

In [ ]:
# Helper function: Produces a plot we will use for a dataset in this demo to visualize errors
def demographics_errors(slope, intercept):
    sample = [[14.7, 33995], [19.1, 61454], [50.7, 71183], [59.5, 105918]]
    demographics.scatter('College%', 'Median Income', alpha=0.5)
    xlims = make_array(5, 75)
    plots.plot(xlims, slope * xlims + intercept, lw=4)
    for x, y in sample:
        plots.plot([x, x], [y, slope * x + intercept], color='r', lw=4)

In [ ]:
# load demographic data 
demographics = Table.read_table('district_demographics2016.csv')
demographics.show(5)

In [ ]:
# select only the 'College%', 'Median Income' columns
demographics = demographics.select('College%', 'Median Income')
demographics.show(5)

In [ ]:
# Calculate the correlation
correlation(demographics, 'College%', 'Median Income')

In [ ]:
# Calculate the slope and intercept
regression_slope = slope(demographics, 'College%', 'Median Income')
regression_intercept = intercept(demographics, 'College%', 'Median Income')
(regression_slope, regression_intercept)

In [ ]:
# create a function that estimates all the predictions (fitted values)
def fitted_values(t, x, y):
    """Return an array of the regressions estimates at all the x values"""
    a = slope(t, x, y)
    b = intercept(t, x, y)
    return a*t.column(x) + b

In [ ]:
# Predicted median incomes (y-hat values)
predicted = fitted_values(demographics, 'College%', 'Median Income')

In [ ]:
# Plot the linear predictions
demographics = demographics.with_column(
    'Linear Prediction', predicted)
demographics.scatter('College%')

In [ ]:
# What are the errors between the actual observations and the predicted values?
actual = demographics.column('Median Income')
errors = actual - predicted
demographics.with_column('Error', errors)

In [ ]:
# Root mean squared error
np.mean(errors ** 2) ** 0.5

In [ ]:
# Visualize errors for estimated line
demographics_errors(regression_slope, regression_intercept)

In [ ]:
# Try any slope, any intercept
demographics_errors(1500, 20000)

In [ ]:
# Try any slope, any intercept
demographics_errors(-1000, 75000)

### Root Mean Square Error ###

In [ ]:
# function to calculate the RMSE for the demographic data
def demographics_rmse(any_slope, any_intercept):
    x = demographics.column('College%')
    y = demographics.column('Median Income')
    estimate = any_slope*x + any_intercept
    return (np.mean((y - estimate) ** 2)) ** 0.5

In [ ]:
# Helper function to visualize the demographic data and print the MSE
def show_demographics_rmse(slope, intercept):
    demographics_errors(slope, intercept)
    rmse = demographics_rmse(slope, intercept)
    print("Root mean squared error:", rmse ** 0.5)

In [ ]:
# show the demographic plot and the RMSE for an arbitrary slope and intercept
show_demographics_rmse(1500, 20000)

In [ ]:
# show the demographic plot and the RMSE for another arbitrary slope and intercept
show_demographics_rmse(-1000, 75000)

In [ ]:
# show the demographic plot and the RMSE for an arbitrary slope and intercept
show_demographics_rmse(500, 20000)

In [ ]:
# show the demographic plot and the RMSE for the slope and intercept we found
show_demographics_rmse(regression_slope, regression_intercept)

## Numerical Optimization

In [ ]:
# An arbitrary function 
def f(x):
    return ((x-2)**2) + 3

# plot the function
x = np.arange(1, 3, 0.1)
y = f(x)
Table().with_columns('x', x, 'y', y).plot('x')

In [ ]:
# find the minimum value of the function 
minimize(f)

### Minimizing RMSE ###

In [ ]:
# RMSE for arbitrary slope and intercept
demographics_rmse(1500, 20000)

In [ ]:
# RMSE for another arbitrary slope and intercept
demographics_rmse(-1000, 75000)

In [ ]:
# Minimize the demographic RMSE
minimize(demographics_rmse)

In [ ]:
# Our regression slope and intercept estimates from earlier
make_array(regression_slope, regression_intercept)

In [ ]:
# The minimum RMSE value
demographics_rmse(regression_slope, regression_intercept)